# Neighbourhood Finder demo

# Imports and helper functions

In [39]:
import json
from random import randint
import os
import json
import lyricwikia
import pickle
import sys
import numpy as np

def artists_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['artist_name'])
    return list(set(all_))

def albums_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['album_name'])
    return list(set(all_))

def tracks_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['track_uri'])
    return list(set(all_))

def track_names_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['track_name'])
    return list(set(all_))

def lyrics_in_playlist(playlist):
    all_ = {}
    info = {}
    ct = 0
    failed = []
    for track in playlist['tracks']:
        track_uri = track['track_uri']
        song_name = track['track_name']
        artist_name = track['artist_name']
        try:
            lyrics = lyricwikia.get_lyrics(artist_name, song_name)
            all_[track_uri] = lyrics
            info[track_uri] = (song_name,artist_name)
        except:
            failed.append(ct)
        ct += 1
    return all_,failed,info

f = open('uri_to_track_info.pckl','rb')
uri_to_name_artist = pickle.load(f)
f.close()

clean_artist_to_songs = {}
for artist in artist_to_songs.keys():
    clean_artist_to_songs[artist] = list(set(artist_to_songs[artist]))

name_artist_to_uri = {}
for uri in uri_to_name_artist.keys():
    tup = uri_to_name_artist[uri]
    name_artist_to_uri[tup] = uri

from gensim.models import Word2Vec
model = Word2Vec.load('model_min5.bin')

# Pick song name by looking up for artist

In [40]:
clean_artist_to_songs['Coldplay']

['The Scientist',
 'Politik',
 'In My Place',
 'True Love - Live At The Enmore Theatre, Sydney',
 "Don't Let It Break Your Heart",
 'Green Eyes',
 'See You Soon',
 'Hurts Like Heaven',
 'Up&Up - Radio Edit',
 'イージー・ツ・プリーズ',
 'Shiver',
 "Now My Feet Won't Touch The Ground",
 'Cemeteries Of London',
 "Everything's Not Lost - Includes Hidden Track 'Life Is For Living'",
 'Speed Of Sound - Live',
 'For You',
 'Paradise',
 'Paradise - Tiësto Remix',
 'Lovers In Japan - Osaka Sun Mix',
 'Trouble',
 'Sparks',
 'A Head Full Of Dreams',
 'One I Love',
 'Parachutes',
 'Kaleidoscope',
 '1.36',
 'A Sky Full of Stars - Robin Schulz Edit',
 'Low',
 'Lost?',
 'Midnight - Live At The Royal Albert Hall, London',
 'The World Turned Upside Down',
 'The Hardest Part',
 'Pour Me - Live At The Hollywood Bowl',
 'Hymn For The Weekend - Seeb Remix',
 'Christmas Lights - Live From Spotify London',
 'Swallowed In The Sea',
 'High Speed',
 'Shiver (Jo Whiley Lunchtime Social)',
 'Hypnotised - EP Mix',
 'Adventur

In [41]:
list_already = []
list_s = [('Warriors','Imagine Dragons'),('The Scientist','Coldplay'),('Immigrant Song','Led Zeppelin'),('T.N.T.','AC/DC')]
song_names = []
artist_names = []
for l in list_s:
    song_names.append(l[0])
    artist_names.append(l[1])
    uri = name_artist_to_uri[l].split(':')[-1]
    list_already.append(uri)
sample = np.zeros(300)
ct = 0
for song in list_already:
    
    track = 'spotify:track:%s'%song
    try:
        ct += 1
        sample += model[track]
    except:
        print(uri_to_name_artist[track],track)
representation = sample/ct

most_sim = model.most_similar([representation],topn=1000)
artists = []
songs = []
for suggestion in most_sim:
    
    songs.append(uri_to_name_artist[suggestion[0]])
    artists.append(uri_to_name_artist[suggestion[0]][1])
#     print(uri_to_name_artist[suggestion[0]])

from collections import Counter
c = Counter(artists)
top_artists = c.most_common(10)

print('Top songs - ')
print(*songs[:25],sep='\n')
print('\n\n\nTop Artisis')
for t in top_artists:
    print(t[0])

most_sim = model.most_similar([representation],topn=1000)
artists = []
songs = []
for suggestion in most_sim:
    
    songs.append(uri_to_name_artist[suggestion[0]])
    artists.append(uri_to_name_artist[suggestion[0]][1])
#     print(uri_to_name_artist[suggestion[0]])

from collections import Counter
c = Counter(artists)
top_artists = c.most_common(14)

print('Top songs - ')
printed = 0
i = 0
while(printed<25):
    song,artist = songs[i]
    if song not in song_names:
        if artist not in artist_names:
            print(song,artist)
            printed += 1
    i += 1
            
print('\n\n\nTop Artisis')
for t in top_artists:
    if t not in artist_names:
        print(t[0])

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


Top songs - 
('Warriors', 'Imagine Dragons')
('T.N.T.', 'AC/DC')
('Immigrant Song', 'Led Zeppelin')
('Monster', 'Imagine Dragons')
('Battle Cry', 'Imagine Dragons')
('Iron Man - Remastered Version', 'Black Sabbath')
('Radioactive', 'Imagine Dragons')
('I’m So Sorry', 'Imagine Dragons')
('Thunderstruck', 'AC/DC')
('Crazy Train - Remastered', 'Ozzy Osbourne')
('Kashmir', 'Led Zeppelin')
('Back In Black', 'AC/DC')
('Shoot to Thrill', 'AC/DC')
('Believer', 'Imagine Dragons')
('Friction', 'Imagine Dragons')
('Hells Bells', 'AC/DC')
('Black Dog', 'Led Zeppelin')
('Who We Are', 'Imagine Dragons')
('In The End', 'Black Veil Brides')
('Highway to Hell', 'AC/DC')
('Bleeding Out', 'Imagine Dragons')
('Seven Nation Army', 'The White Stripes')
('Smoke On the Water', 'Deep Purple')
('Dirty Deeds Done Dirt Cheap', 'AC/DC')
('Dream', 'Imagine Dragons')



Top Artisis
Imagine Dragons
Fall Out Boy
Linkin Park
AC/DC
Three Days Grace
Muse
Coldplay
Panic! At The Disco
Led Zeppelin
My Chemical Romance
Top s

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
